# This notebook compares our system to Kallisto, using KL divergence on generated data

In [ ]:
import csv
import seaborn as sns
from ExplainableMaximumLikelihoodCalculator import ExplainableMaximumLikelihoodCalculator
import pysam
from Bio import SeqIO, Seq, SeqRecord, pairwise2
from Bio.pairwise2 import format_alignment
from BamFileUtils import getListOfReadsFromBamFile, getListOfReadsFromFastaFile, create_fasta_from_list_of_reads, getKallistoAbundance
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import math
import shap
from scipy.stats import chisquare
from scipy.special import rel_entr
fileNameSapiens =  "data/simulatedData/human_chinese_AF346973_500samples.fas"
fileNameNeanderthals =  "data/simulatedData/Neanderthal_Goyet_KX198085_500samples.fas"
fileNameDenisovans =  "data/simulatedData/denisova_kx663333_500samples.fas"
import time

neanderthals_500_generated = getListOfReadsFromFastaFile(fileNameNeanderthals)
sapiens_500_generated = getListOfReadsFromFastaFile(fileNameSapiens)
denisovan_500_samples = getListOfReadsFromFastaFile(fileNameDenisovans)
path_to_frequencies_table = "data/substitution_matrix.tsv"

In [ ]:
# !conda install -c bioconda kallisto

In [ ]:
number_of_sapienses = 85
number_of_neanderthals = 85
number_of_denisovans = 85

total_length = number_of_sapienses + number_of_neanderthals + number_of_denisovans
all_indexes = [i for i in range(total_length)]
neanderthal_indexes = [i for i in all_indexes if i<number_of_neanderthals]
sapienses_indexes = [i for i in all_indexes if i>= number_of_neanderthals and i < number_of_sapienses + number_of_neanderthals]
denisovans_indexes = [i for i in all_indexes if i>= number_of_sapienses + number_of_neanderthals]

simulated_reads = neanderthals_500_generated[:number_of_neanderthals] + sapiens_500_generated[:number_of_sapienses] + denisovan_500_samples[:number_of_denisovans]

In [ ]:
sapiens_reference_file_names = [
                    "data/reference_files/human_AF346981_French.fa",
                     "data/reference_files/human_AY195760_Korea.fa",
                      "data/reference_files/human_AY882416_Ethiopia.fa",
                      "data/reference_files/human_AY963586_Italian.fa",
                      "data/reference_files/human_AY195781_Caucasian.fa",
                      "data/reference_files/human_AY195757_Iraqi-Israeli.fa",
                      "data/reference_files/human_AY195749_NativeAmerican.fa"]
neanderthals_reference_file_names = [
                            "data/reference_files/neanderthal_mezmaiskaya1_FM865411.fa",
                           "data/reference_files/Neanderthal_Altai_KC879692.fa",
                           "data/reference_files/Neanderthal_Denisova11_full_mtDNA_KU131206.fa",
                           "data/reference_files/Neanderthal_Spy_94a_MG025538.fa",
                            "data/reference_files/Neanderthal_Vindija33.16_AM948965.fa",
                            "data/reference_files/Neanderthal_Vindija33.19_KJ533545.fa",]
denisovan_reference_file_names = [  
                        "data/reference_files/Denisova_MT576653.1.fa",
                        "data/reference_files/Denisova_MT576652.1.fa",
                        "data/reference_files/Denisova_4_FR695060.fa",
                        "data/reference_files/Denisova_8_KT780370.fa",
                        "data/reference_files/Denisova_manual_phalanx_NC_013993.fa",
                        "data/reference_files/Denisova_MT576651.1.fa"]

In [ ]:
!kallisto index -i transcripts_all_refs.idx data/reference_files/Denisova_MT576651.1.fa data/reference_files/Denisova_manual_phalanx_NC_013993.fa data/reference_files/Denisova_8_KT780370.fa data/reference_files/Denisova_4_FR695060.fa data/reference_files/Denisova_MT576652.1.fa data/reference_files/Denisova_MT576653.1.fa data/reference_files/Neanderthal_Spy_94a_MG025538.fa data/reference_files/Neanderthal_Vindija33.19_KJ533545.fa data/reference_files/Neanderthal_Vindija33.16_AM948965.fa data/reference_files/Neanderthal_Denisova11_full_mtDNA_KU131206.fa data/reference_files/Neanderthal_Altai_KC879692.fa data/reference_files/neanderthal_mezmaiskaya1_FM865411.fa data/reference_files/human_AY195749_NativeAmerican.fa data/reference_files/human_AY195757_Iraqi-Israeli.fa data/reference_files/human_AY195781_Caucasian.fa data/reference_files/human_AF346981_French.fa data/reference_files/human_AY195760_Korea.fa data/reference_files/human_AY882416_Ethiopia.fa data/reference_files/human_AY963586_Italian.fa

In [ ]:
maximum_likelihood_calculator_d_1 = ExplainableMaximumLikelihoodCalculator(simulated_reads,
                                                        ref_neanderthal_file_names=neanderthals_reference_file_names,
                                                        ref_sapien_file_names=sapiens_reference_file_names,
                                                        ref_denisovan_file_names=denisovan_reference_file_names,
                                                        path_to_substitution_matrix=path_to_frequencies_table,
                                                        number_of_jobs=-1)

### Run maximum likelihood on all the data

In [ ]:
maximum_likelihood_calculator_d_1.estimate_species_proportions(100)

### Run Kallisto on all the data

In [ ]:
create_fasta_from_list_of_reads("sample_for_kalisto.fas", simulated_reads)
!kallisto quant -i transcripts_all_refs.idx -o output -b 100 --single sample_for_kalisto.fas -l 75 -s 0.02 &>/dev/null
result_kalisto = getKallistoAbundance()

In [ ]:
def generate_sample(sample_size):
    species = random.randint(0,2)
    
    tenth = sample_size //10
    number_of_neanderthals = tenth
    number_of_sapiens = tenth
    number_of_denisovans = tenth
    
    if (species ==0):
        number_of_sapiens = sample_size - tenth*2
    if (species ==1):
        number_of_neanderthals = sample_size- tenth*2
    if (species == 2):
        number_of_denisovans = sample_size- tenth*2
    return (number_of_sapiens, number_of_neanderthals, number_of_denisovans)

In [ ]:
sample_sizes = [10,20,30,40,50,60,70,80,90,100]
print(sample_sizes)
indexes = [i for i in range(total_length)]
number_of_trials_per_sample_size = 200

data_kl = []
distances = []

for sample_size in sample_sizes:
    print(sample_size)
    number_of_samples = 0
    while (number_of_samples < number_of_trials_per_sample_size):
        if (number_of_samples%10 == 0):
            print(sample_size, number_of_samples)
        (sapien_sample_size, neanderthal_sample_size, denisovan_sample_size) = generate_sample(sample_size)
        neanderthal_sample = random.sample(neanderthal_indexes, neanderthal_sample_size)
        sapien_sample = random.sample(sapienses_indexes, sapien_sample_size)
        denisovan_sample = random.sample(denisovans_indexes, denisovan_sample_size)
        sample = neanderthal_sample + sapien_sample + denisovan_sample
        neanderthals_in_sample = neanderthal_sample_size/sample_size
        sapiens_in_sample = sapien_sample_size/sample_size
        denisovans_in_sample = denisovan_sample_size/sample_size
        
        expected_result = np.asarray([sapiens_in_sample, neanderthals_in_sample, denisovans_in_sample])
        
        sample_to_run_kalisto_on = [simulated_reads[k] for k in indexes if k in sample]
        create_fasta_from_list_of_reads("sample_for_kalisto.fas", sample_to_run_kalisto_on)
        !kallisto quant -i transcripts_all_refs.idx -o output -b 100 --single sample_for_kalisto.fas -l 75 -s 0.02 &>/dev/null
        result_kalisto = getKallistoAbundance()
        
        result_likeli_calc = maximum_likelihood_calculator_d_1.calc_maximum_likelihood_on_subset(sample,100).values[0]

        distance_kalisto = np.absolute(result_kalisto - expected_result)
        distance_likeli = np.absolute(result_likeli_calc - expected_result)

        kl_divergence_kalisto = sum(rel_entr(result_kalisto, expected_result))
        kl_divergence_likeli = sum(rel_entr(result_likeli_calc, expected_result))

        data_kl.append((sample_size, kl_divergence_kalisto, "Kallisto"))
        data_kl.append((sample_size, kl_divergence_likeli, "Maximum Likelihood Calculator"))

        distances.append((sample_size, distance_kalisto, "Kallisto"))
        distances.append((sample_size, distance_likeli, "Maximum Likelihood Calculator"))

        number_of_samples+=1


In [ ]:
distances_averaged = [(i[0], np.mean(i[1]), i[2]) for i in distances]
data_kallisto = [i for i in distances_averaged if i[2] == "Kallisto"]
data_maximum_likelihood = [i for i in distances_averaged if i[2] == "Maximum Likelihood Calculator"]

In [ ]:
data_averaged_likeli = []
data_averaged_kallisto = []
data_averaged_partial= []

for size in sample_sizes:
    kali = [i[1] for i in data_kallisto if i[0] == size]
    likeli_calc_list = [i[1] for i in data_maximum_likelihood if i[0] == size]
    average_kali = np.mean(kali)
    average_likeli = np.mean(likeli_calc_list)
    data_averaged_kallisto.append((size, average_kali))
    data_averaged_likeli.append((size, average_likeli))


In [ ]:
data_kallisto = [i for i in data_kl if i[2] == "Kallisto"]
data_maximum_likelihood = [i for i in data_kl if i[2] == "Maximum Likelihood Calculator"]


In [ ]:
sample_sizes = [10*i for i in range(100) if i > 0 and 10*i < 101]

data_averaged_likeli = []
data_averaged_kallisto = []

for size in sample_sizes:
    kali = [i[1] for i in data_kallisto if i[0] == size]
    likeli_calc_list = [i[1] for i in data_maximum_likelihood if i[0] == size]
    average_kali = np.mean(kali)
    average_likeli = np.mean(likeli_calc_list)
    
    data_averaged_kallisto.append((size, average_kali))
    data_averaged_likeli.append((size, average_likeli))

# create data
x = sample_sizes
y_kalisto = [i[1] for i in data_averaged_kallisto]
y_likeli = [i[1] for i in data_averaged_likeli]
  
plt.rcParams.update({'font.size': 15})
plt.plot(x, y_kalisto, label = "Kallisto", linewidth=3, color="green", linestyle="dashed")
plt.plot(x, y_likeli, label = "Algorithm 1", linewidth=3, color="brown")
plt.xlabel("Dataset size")
plt.ylabel("KL-Divergence from ground truth")
plt.legend()

plt.rcParams.update({'font.size': 12})

plt.text(x[-1]-5, y_kalisto[-1]+0.01, str(y_kalisto[-1])[:4])
plt.savefig("sampleSizeToKLDivergenceKallistoComparedToexML.png")